# Análise de investimentos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
import yfinance 

yfinance.pdr_override()
end = "2022-12-31"

In [ ]:
procter_and_gamble = pdr.get_data_yahoo('PG', start='1995-01-01', end=end)

# opção do csv baixado da página a yahoo
# procter_and_gamble = pd.read_csv('../data/PG.csv')

In [ ]:
procter_and_gamble.head()

In [ ]:
procter_and_gamble.tail()

In [ ]:
pg = procter_and_gamble[["Adj Close"]].copy().reset_index()
pg['Date'] = pd.to_datetime(pg['Date'])
pg['ano'] = pg.Date.dt.year
pg['mes'] = pg.Date.dt.month
pg = pg[['Date', 'ano', 'mes', 'Adj Close']]

pg

## Retorno simples 
Calculado sobre o preço de fechamento

$$\text{retorno simples} = \frac{P_1-P_0}{P_0} = \frac{P_1}{P_0}-1$$ 

In [ ]:
pg

In [ ]:
pg['retorno_simples'] = (pg['Adj Close']/pg['Adj Close'].shift(1)) - 1

In [ ]:
pg[['Date', 'retorno_simples']].plot(x='Date', y='retorno_simples', figsize=(8, 5))
plt.title("Retornos diários da ações PG entre 01-01-1995 até 23-03-2017")
plt.plot(pg.Date, len(pg.Date)*[0], '--')
plt.show()

In [ ]:
pg[['Date', 'Adj Close']].plot(x='Date', y='Adj Close',figsize=(8, 5))
plt.title("Preço de fechamento para ações PG entre 01-01-1995 até 23-03-2017")
plt.show()

In [ ]:
avg_returns = pg['retorno_simples'].mean()
avg_returns

In [ ]:
avg_returns_year = pg['retorno_simples'].mean() * 250
print(f'retorno anual médio {round(avg_returns_year*100, 3)} %')

## Retorno logarítmo

Lembrando que fazemos uma análise de um único ativo. Nesse caso é mais interessante analisar o retorno logarítmo.

$$\text{retorno log} = \ln\left(\frac{P_{t}}{P_{t-1}}\right)$$

In [ ]:
pg['retorno_log'] = np.log(pg['Adj Close']/pg['Adj Close'].shift(1))

In [ ]:
pg

In [ ]:
pg[['Date', 'retorno_log']].plot(x='Date', y='retorno_log',figsize=(8, 5))
plt.title("Retornos (log) diários da ações PG entre 01-01-1995 até 23-03-2017")
plt.plot(pg.Date, len(pg.Date)*[0], '--')
plt.show()

## Retorno de multiplos investimentos

Vamos considerar um portifólio simples, onde os quatro ativos tem pesos iguais, e calcular as suas respectivas taxas de retorno histórico.

\begin{array}{|c|c|c|}
\hline
\text{Ativo} & \text{Taxa média anual de retorno(\%)} & Peso \\
\hline
\text{P\&G} &   0.318524 & 0.25 \\
\text{MSFT} &  0.153581 & 0.25\\
\text{GE} &  0.116717 & 0.25\\
\text{F} & 0.126224 & 0.25\\
\hline
\end{array}

In [ ]:
# portifólio com ações da P&G,Microsoft, Ford, General Electrics
tickers = ["PG", 'MSFT', 'F', 'GE'] 
preco_fechamento = pdr.get_data_yahoo(tickers, start='1995-01-01', end=end)['Adj Close']

In [ ]:
acoes = preco_fechamento.copy()
acoes = acoes.rename(
    columns={'F': 'ford', 
             'GE': 'ge',
             'MSFT': 'microsoft', 
             'PG': 'pg'}
)

acoes.head()

### Retornos simples - indicados para avaliar uma carteira de investimentos diversificados
 - Normalizando os valores - comparar do mesmo ponto inicial- data base
$$\frac{P_{t}}{P_{0}}*100$$


In [ ]:
acoes.iloc[0]

In [ ]:
acoes = (acoes/acoes.iloc[0])*100

In [ ]:
acoes.plot(figsize=(12, 8))
plt.show()

In [ ]:
# retorno simples para um portifólio
retornos = (acoes/acoes.shift(1))-1

In [ ]:
retornos.head()

In [ ]:
pesos = np.array(4*[0.25])
pesos

In [ ]:
# estimando o retorno anual médio
retorno_anual = retornos.mean() *250
retorno_anual = retorno_anual.reset_index().rename(
    columns={'index':'ativo',
             0:'retorno médio anual'
})

retorno_anual

In [ ]:
print(f"""Retorno anual da carteira de investimentos com pesos iguais
{round(np.dot(retorno_anual['retorno médio anual'], pesos) * 100, 3)} %""")

In [ ]:
pesos_2 = np.array([0.4,0.4, 0.15, 0.05])
print(f"""Retorno anual da carteira de investimentos com pesos diferentes
{round(np.dot(retorno_anual['retorno médio anual'], pesos_2) * 100, 3)} %""")

# Índices de ações- calculando taxas de retorno desses indices

In [ ]:
# S&P500, NASDAQ, indice alemão DAX, footsie
index_tickers = ['^GSPC', '^IXIC', '^GDAXI', '^FTSE']

indices = pdr.get_data_yahoo(index_tickers, start='1997-01-01', end=end)['Adj Close']
indices = indices.rename(
    columns={'^GSPC': 'S&P500',
             '^IXIC': 'NASDAQ',
             '^GDAXI': 'DAX',
             '^FTSE': 'Footsie'}
)

In [ ]:
indices.head()

In [ ]:
indices.tail()

In [ ]:
# normalizando para o valor base
(100*(indices/indices.iloc[0])).plot(figsize=(15, 6))
plt.axvline(x=pd.to_datetime("2000-01-01"), color='b', label='bolha da internet nos anos 2000')
plt.axvline(x=pd.to_datetime("2007-08-30"), color='b', label='crise de 2008')
plt.axvline(x=pd.to_datetime("2011-06-01"), color='b', label='crise de 2011')
plt.axvline(x=pd.to_datetime("2020-03-01"), color='b', label='pandemia COVID-19')

# place legend outside
plt.legend(bbox_to_anchor=(1.0, 1), loc='best')
plt.show()

In [ ]:
# retornos
retorno_indices = indices/indices.shift(1)-1
retorno_indices.tail()

In [ ]:
# Taxas positivas de retornos
retorno_anual_indices = retorno_indices.mean()*250
retorno_anual_indices = retorno_anual_indices.reset_index().rename(
    columns={'index': 'ativo',
             0: 'retorno anual médio'})

retorno_anual_indices

# Comparando o preço de fechamento de uma ação com os indices de mercado

In [ ]:
tickers = ['PG', '^GSPC', '^DJI']

indices_comparados = pdr.get_data_yahoo(tickers, start='1997-01-01', end=end)['Adj Close']
indices_comparados = indices_comparados.rename(
    columns={'^GSPC': 'S&P500',
             '^DJI': 'DowJones'}
)

In [ ]:
indices_comparados.tail()

In [ ]:
# normalizando para o valor base
indices_comparados = 100*(indices_comparados/indices_comparados.iloc[0])
                      
indices_comparados.plot(figsize=(15, 6))
plt.axvline(x=pd.to_datetime("2000-01-01"), color='red', label='bolha da internet nos anos 2000')
plt.axvline(x=pd.to_datetime("2007-08-30"), color='b', label='crise de 2008')
plt.axvline(x=pd.to_datetime("2011-06-01"), color='pink', label='crise de 2011')
plt.axvline(x=pd.to_datetime("2020-03-01"), color='green', label='pandemia COVID-19')

# place legend outside
plt.legend(bbox_to_anchor=(1.0, 1), loc='best')
plt.show()

In [ ]:
indices_comparados

# Calculando o risco de uma ação

In [ ]:
# comparar os históricos e o risco entre dois ativos de mesmo negócio Beiersdorf e P&G
tickers = ['PG', 'BEI.DE']

dados_hist = pdr.get_data_yahoo(tickers, start='2007-01-01', end="2022-12-31")['Adj Close']
dados_hist = dados_hist.rename(
    columns={'PG': 'P&G',
             'BEI.DE': 'Beiersdorf'}
)

In [ ]:
dados_hist.head()

In [ ]:
dados_hist.tail()

In [ ]:
dados_hist = dados_hist.dropna()

In [ ]:
# normalizando
(100*(dados_hist/dados_hist.iloc[0])).plot(figsize=(15, 6))
plt.axvline(x=pd.to_datetime("2007-08-30"), color='b', label='crise de 2008')
plt.axvline(x=pd.to_datetime("2011-06-01"), color='b', label='crise de 2011')
plt.axvline(x=pd.to_datetime("2020-03-01"), color='b', label='pandemia COVID-19')

# place legend outside
plt.legend(bbox_to_anchor=(1.0, 1), loc='best')
plt.show()

### Calculando risco

In [ ]:
sec_returns = np.log(dados_hist/dados_hist.shift(1))

In [ ]:
# media anual
avg_sec_returns = (sec_returns.mean() * 250).reset_index().rename(
    columns={'index': 'ativo',
             0: 'media anual de retorno'})
avg_sec_returns

In [ ]:
# risco - calculao do desvio padrão
risco = ((sec_returns.std() * 250)**0.5).reset_index().rename(
    columns={'index': 'ativo',
             0: 'risco'})
risco

# Correlação